In [1]:
file_path = "Bangla-NER-Splitted-Dataset.json"

In [2]:
import json
  
# Opening JSON file
f = open(file_path, mode="r", encoding="utf-8")
  
# returns JSON object as 
# a dictionary
json_data = json.load(f)

In [3]:
json_data.keys()

dict_keys(['test', 'train', 'validation'])

In [4]:
train_json_data = json_data['train']
val_json_data = json_data['validation']
test_json_data = json_data['test']

In [5]:
import pandas as pd
import re
import numpy as np

train_df = pd.DataFrame(columns=['ner_tags', 'tokens'])
val_df = pd.DataFrame(columns=['ner_tags', 'tokens'])
test_df = pd.DataFrame(columns=['ner_tags', 'tokens'])

In [6]:
def read_json_file(json_file, df):
    token_docs = []
    tag_docs = []

    for idx,doc in enumerate(json_file):
        tokens = doc['sentence']
        tags = doc['iob_tags']
        token_docs.append(tokens)
        tag_docs.append(tags)
        df.loc[idx] = pd.Series({'ner_tags':tags, 'tokens':tokens})
    
    return df, token_docs, tag_docs

In [7]:
train_df,_,tag_docs = read_json_file(train_json_data, train_df)

In [8]:
val_df,_,_ = read_json_file(val_json_data, val_df)

In [9]:
test_df,_,_ = read_json_file(test_json_data, val_df)

In [10]:
train_df[:1]

,ner_tags,tokens
0,"[O, O, O, B-PER, B-PER, I-PER, I-PER, O, O, O,...","[ত্রাণ, ও, সমাজকল্যাণ, সম্পাদক, সুজিত, রায়, নন..."


In [11]:
val_df[:1]

,ner_tags,tokens
0,"[O, O, O, B-LOC, O, O, O, O]","[৫%, তার, চাইতে, পশ্চিমোরে, এর, সাক্ষরতার, হার..."


In [12]:
test_df[:1]

,ner_tags,tokens
0,"[O, O, O, B-LOC, O, O, O, O]","[৫%, তার, চাইতে, পশ্চিমোরে, এর, সাক্ষরতার, হার..."


In [13]:
from datasets import Dataset, DatasetDict

train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)
test_ds = Dataset.from_pandas(test_df)

In [14]:
datasets = DatasetDict()
datasets['train'] = train_ds
datasets['validation'] = val_ds
datasets['test'] = test_ds

In [15]:
datasets["train"][3]["tokens"]

['তিনি',
 'বলছিলেন',
 'সবচেয়ে',
 'বড়',
 'কথা',
 'উনি',
 'খুব',
 'ভালো',
 'মানুষ',
 'ছিলেন']

In [16]:
datasets["train"][3]["ner_tags"]

['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O']

In [17]:
# create set from list
expanded_tag_docs = []

for tags in tag_docs:
    for tag in tags:
        expanded_tag_docs.append(tag)

In [18]:
unique_tags = set(expanded_tag_docs)
unique_tags

{'B-LOC', 'B-OBJ', 'B-ORG', 'B-PER', 'I-LOC', 'I-OBJ', 'I-ORG', 'I-PER', 'O'}

In [19]:
def assign_label(examples):
    mapping = {'B-LOC':0, 'B-OBJ':1, 'B-ORG':2, 'B-PER':3, 'I-LOC':4, 'I-OBJ':5, 'I-ORG':6, 'I-PER':7, 'O':8}
    ner_labels = []
    for example in examples["ner_tags"]:
        ner_labels.append(mapping[example])
    examples["ner_labels"] = ner_labels
    return examples

In [20]:
datasets = datasets.map(assign_label)

  0%|          | 0/64155 [00:00<?, ?ex/s]

  0%|          | 0/3565 [00:00<?, ?ex/s]

  0%|          | 0/3565 [00:00<?, ?ex/s]

In [22]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert", keep_accents=True)

In [23]:
inputs = tokenizer(datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 '▁ত্রাণ',
 '▁ও',
 '▁সমাজকল্যাণ',
 '▁সম্পাদক',
 '▁সু',
 'জিত',
 '▁রায়',
 '▁নন্দী',
 '▁প্রমুখ',
 '▁সংবাদ',
 '▁সম্মেলনে',
 '▁উপস্থিত',
 '▁ছিলেন',
 '[SEP]']

In [24]:
inputs.word_ids()

[None, 0, 1, 2, 3, 4, 4, 5, 6, 7, 8, 9, 10, 11, None]

In [25]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [26]:
labels = datasets["train"][0]["ner_labels"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[8, 8, 8, 3, 3, 7, 7, 8, 8, 8, 8, 8]
[-100, 8, 8, 8, 3, 3, 4, 7, 7, 8, 8, 8, 8, 8, -100]


In [27]:
datasets["train"][1]

{'ner_tags': ['O',
  'O',
  'B-PER',
  'B-OBJ',
  'O',
  'O',
  'B-PER',
  'O',
  'O',
  'B-PER',
  'O',
  'O',
  'B-PER',
  'O',
  'B-OBJ',
  'O'],
 'tokens': ['পরিকল্পনা',
  'অনুযায়ী',
  'তারা',
  'বাসায়',
  'ঢুকে',
  'দুই',
  'অতিথিকে',
  'নগ্ন',
  'করে',
  'তাদের',
  'মাঝখানে',
  'এক',
  'ছাত্রীকে',
  'বসিয়ে',
  'ছবি',
  'তোলেন'],
 '__index_level_0__': 1,
 'ner_labels': [8, 8, 3, 1, 8, 8, 3, 8, 8, 3, 8, 8, 3, 8, 1, 8]}

In [28]:
mapping = {'B-LOC':0, 'B-OBJ':1, 'B-ORG':2, 'B-PER':3, 'I-LOC':4, 'I-OBJ':5, 'I-ORG':6, 'I-PER':7, 'O':8}
label_names = list()
for k, v in mapping.items():
    label_names.append(k)

label_names

['B-LOC', 'B-OBJ', 'B-ORG', 'B-PER', 'I-LOC', 'I-OBJ', 'I-ORG', 'I-PER', 'O']

In [29]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True, max_length=128
    )
    all_labels = examples["ner_labels"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [30]:
tokenized_datasets = datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=datasets["train"].column_names,
)

  0%|          | 0/65 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [31]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 64155
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3565
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3565
    })
})

In [32]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [33]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

You're using a AlbertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100,    8,    8,    8,    3,    3,    4,    7,    7,    8,    8,    8,
            8,    8, -100, -100, -100, -100, -100],
        [-100,    8,    8,    3,    1,    8,    8,    3,    4,    8,    8,    3,
            8,    8,    3,    8,    1,    8, -100]])

In [34]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 8, 8, 8, 3, 3, 4, 7, 7, 8, 8, 8, 8, 8, -100]
[-100, 8, 8, 3, 1, 8, 8, 3, 4, 8, 8, 3, 8, 8, 3, 8, 1, 8, -100]


In [35]:
import evaluate

metric = evaluate.load("seqeval")

In [36]:
labels = datasets["train"][0]["ner_labels"]
labels = [label_names[i] for i in labels]
labels

['O', 'O', 'O', 'B-PER', 'B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O']

In [37]:
predictions = labels.copy()
predictions[2] = 'B-PER'
metric.compute(predictions=[predictions], references=[labels])

{'PER': {'precision': 0.6666666666666666,
  'recall': 1.0,
  'f1': 0.8,
  'number': 2},
 'overall_precision': 0.6666666666666666,
 'overall_recall': 1.0,
 'overall_f1': 0.8,
 'overall_accuracy': 0.9166666666666666}

In [38]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [39]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [40]:
id2label

{0: 'B-LOC',
 1: 'B-OBJ',
 2: 'B-ORG',
 3: 'B-PER',
 4: 'I-LOC',
 5: 'I-OBJ',
 6: 'I-ORG',
 7: 'I-PER',
 8: 'O'}

In [41]:
label2id

{'B-LOC': 0,
 'B-OBJ': 1,
 'B-ORG': 2,
 'B-PER': 3,
 'I-LOC': 4,
 'I-OBJ': 5,
 'I-ORG': 6,
 'I-PER': 7,
 'O': 8}

In [42]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    "ai4bharat/indic-bert",
    id2label=id2label,
    label2id=label2id,
)

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForTokenClassification: ['predictions.dense.bias', 'predictions.decoder.bias', 'predictions.LayerNorm.bias', 'sop_classifier.classifier.bias', 'predictions.decoder.weight', 'predictions.dense.weight', 'predictions.LayerNorm.weight', 'predictions.bias', 'sop_classifier.classifier.weight']
- This IS expected if you are initializing AlbertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and a

In [43]:
model.config.num_labels

9

In [44]:
#  disable weights and biases logging
import os
os.environ["WANDB_DISABLED"] = "true"

In [45]:
from transformers import TrainingArguments
batch_size = 16
# batch_size = 32

args = TrainingArguments(
    "models/ner",
    report_to = None,
    logging_dir= None,
    save_strategy="no",
    evaluation_strategy="epoch",
    #save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [46]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

In [47]:
import transformers
trainer.remove_callback(transformers.integrations.TensorBoardCallback)

In [48]:
trainer.evaluate()

{'eval_loss': 2.199763536453247,
 'eval_precision': 0.03795969651572996,
 'eval_recall': 0.14457942205043253,
 'eval_f1': 0.06013166960116358,
 'eval_accuracy': 0.08695105861657347,
 'eval_runtime': 9.8761,
 'eval_samples_per_second': 360.971,
 'eval_steps_per_second': 22.58}

In [49]:
trainer.train()

C:\Users\arifa\anaconda3\envs\pytorch_env\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.380300,0.391942,0.669475,0.533867,0.594030,0.869997
2,0.334200,0.365676,0.694409,0.549788,0.613694,0.877772
3,0.301600,0.366442,0.676518,0.579330,0.624163,0.877584


TrainOutput(global_step=12030, training_loss=0.36723918581841297, metrics={'train_runtime': 915.515, 'train_samples_per_second': 210.226, 'train_steps_per_second': 13.14, 'total_flos': 283898589880422.0, 'train_loss': 0.36723918581841297, 'epoch': 3.0})

In [49]:
trainer.save_model()

In [50]:
# trainer.args.num_train_epochs = trainer.args.num_train_epochs + 1
# trainer.train("models/ner/checkpoint-2005")

In [50]:
import gc, torch
gc.collect()
torch.cuda.empty_cache()

In [51]:
trainer.evaluate(tokenized_datasets["test"])

{'eval_loss': 0.3664419949054718,
 'eval_precision': 0.6765180010746911,
 'eval_recall': 0.5793300202466409,
 'eval_f1': 0.6241634029051609,
 'eval_accuracy': 0.8775837155242822,
 'eval_runtime': 6.3844,
 'eval_samples_per_second': 558.393,
 'eval_steps_per_second': 34.929,
 'epoch': 3.0}